# **Montar Google Drive**

En la siguiente celda se muestra el código correspondiente al montaje de archivos que se encuentran en Google Drive. Para ello se solicita la autorización para poder acceder para permitir el montaje en la máquina virtual.


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import numpy as np
import pandas as pd

# **Acceso al corpus**

El dataset con el que se trabaja contiene una colección de 4500 documentos extraidos de Twitter con ninguna ninguna temática común. 

Los documentos que forman este corpus ya ha pasado por un proceso de limpieza en el se ha:

 

1. Eliminado enlaces, menciones, hashtags y retweets
2. Reducir caracteres duplicados
3. Suprimir risas (hahaha, jajaja) contemplando que puedan ser con cualquier vocal
4. Eliminado tildes o caracteres raros que se encuentren en las letras.
5. Descartar cualquier caracter que no sean letras
6. Se sustituyen los tabuladores por espacios en blanco
7. Si hay varios espacios en blanco se sustituyen por un solo espacio.
8. Quitar espacios al comienzo del tweet y al final
9. Eliminar saltos de línea que puedan existir en cualquier parte de la cadena.
10. Elimiar stopwords
11. Aplicar stemming


Así mismo, se han eliminado todas las columnas que no eran relevantes, dando como resultado un corpus formado por dos columnas. La primera representa el sentimiento asociado a un documento y la segunda alberga cada uno de los tweets.

Las polaridades contempladas son dos únicamente; 0 para el sentimiento negativo y 4 para el positivo.

In [ ]:
corpus = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/sentiment140.csv")
corpus

,Sentiment,Tweet
0,4,realis use twitter often coz atm look like lea...
1,0,work today
2,0,still sleep
3,0,heavi week ahead heavi cold match
4,0,thunder much cute hair quot anoth music festiv...
...,...,...
44995,4,lunch bff
44996,0,sorri hear dog
44997,4,goodnight world
44998,0,make regret live state


Para que dure menos el proceso se va a reducir el corpus a 1000 documentos.

In [ ]:
reduced_corpus = corpus.iloc[:1000]
reduced_corpus

,Sentiment,Tweet
0,4,realis use twitter often coz atm look like lea...
1,0,work today
2,0,still sleep
3,0,heavi week ahead heavi cold match
4,0,thunder much cute hair quot anoth music festiv...
...,...,...
995,0,im sure realli andi
996,0,dinner ok mussel appet follow chicken stew quo...
997,0,secretgarden gotten porn spammer check follow ...
998,0,nap interrupt mani time today go japanes rent


Se utiliza la función pandas.DataFrame.***astype()*** para convertir una columna particular del corpus en el tipo de dato específico que se indique entre parentesis.



In [ ]:
reduced_corpus["Tweet_split"] = reduced_corpus["Tweet"].astype(str)
reduced_corpus

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Sentiment,Tweet,Tweet_split
0,4,realis use twitter often coz atm look like lea...,realis use twitter often coz atm look like lea...
1,0,work today,work today
2,0,still sleep,still sleep
3,0,heavi week ahead heavi cold match,heavi week ahead heavi cold match
4,0,thunder much cute hair quot anoth music festiv...,thunder much cute hair quot anoth music festiv...
...,...,...,...
995,0,im sure realli andi,im sure realli andi
996,0,dinner ok mussel appet follow chicken stew quo...,dinner ok mussel appet follow chicken stew quo...
997,0,secretgarden gotten porn spammer check follow ...,secretgarden gotten porn spammer check follow ...
998,0,nap interrupt mani time today go japanes rent,nap interrupt mani time today go japanes rent


Se debe aplicar ***split()*** en los documentos porque Word2Vec necesita como entrada un conjunto de frases que sean iterables. 

Esta función está asociada a los tipos de datos ***string*** que devuelve una lista formada por cada una de las palabras que forman una frase u oración.

In [ ]:
reduced_corpus["Tweet_split"] = reduced_corpus["Tweet_split"].str.split(" ")
reduced_corpus

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Sentiment,Tweet,Tweet_split
0,4,realis use twitter often coz atm look like lea...,"[realis, use, twitter, often, coz, atm, look, ..."
1,0,work today,"[work, today]"
2,0,still sleep,"[still, sleep]"
3,0,heavi week ahead heavi cold match,"[heavi, week, ahead, heavi, cold, match]"
4,0,thunder much cute hair quot anoth music festiv...,"[thunder, much, cute, hair, quot, anoth, music..."
...,...,...,...
995,0,im sure realli andi,"[im, sure, realli, andi]"
996,0,dinner ok mussel appet follow chicken stew quo...,"[dinner, ok, mussel, appet, follow, chicken, s..."
997,0,secretgarden gotten porn spammer check follow ...,"[secretgarden, gotten, porn, spammer, check, f..."
998,0,nap interrupt mani time today go japanes rent,"[nap, interrupt, mani, time, today, go, japane..."


In [ ]:
reduced_corpus.shape

(1000, 3)

# **Representación del texto como un vector: Word2Vec**
El procesamiento automático de textos requiere adaptar la presentación textual de un documento a un formato de entrada válido para distintos algoritmos. La forma que se utiliza en este ejemplo se conoce como ***"word2Vec"***, un tipo de representación donde cada palabra se transforma en un vector .

In [ ]:
import gensim

tokenized_tweet = reduced_corpus["Tweet_split"]

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            window=5, # tamaño de la ventana de contexto
            min_count=2, # ignora todas las palabras con una frecuencia total inferior a 2.                                 
            sg = 1, # 1 para el modelo skip-gram
            hs = 0, # 1 selecciona softmax jerárquico; 0 elige muestreo negativo.
            negative = 10, # para el muestreo negativo
            workers= 32, # indica el número de hilos con los que se trabajará durante el entrenamiento del modelo
            seed = 34,
            size=300
) 

# model.build_vocab(tokenized_tweet)

model_w2v.train(tokenized_tweet, total_examples= len(reduced_corpus["Tweet_split"]), epochs=20 ) # numero de iteraciones


(88865, 136840)

Tamaño del vector asociada a cada una de las palabras, en caso de querer definir la longitud se debería indicar el número en la variable ***size*** 

In [ ]:
size_vector = len(model_w2v.wv['goodnight'])
size_vector

100

In [ ]:
model_w2v.wv['goodnight'].shape

(100,)

In [ ]:
model_w2v.wv['goodnight']

array([-0.04899945,  0.05775623, -0.01200921,  0.13747592,  0.01744397,
       -0.06019304,  0.09607932, -0.1320978 ,  0.2306533 , -0.08154952,
        0.11864791, -0.03696945, -0.16695537, -0.00541155, -0.08558234,
       -0.00783106,  0.21356264,  0.17416938, -0.16976668,  0.21633425,
        0.1487263 , -0.16255456,  0.3605209 ,  0.13222001, -0.1486412 ,
       -0.2048432 , -0.21100844,  0.10778131,  0.12574403,  0.19410014,
       -0.10554123,  0.160498  , -0.02122359,  0.03678202,  0.00498264,
       -0.09373675,  0.30038366, -0.29669207,  0.07080056, -0.0845767 ,
        0.18550898,  0.17228207,  0.1398607 , -0.01778615, -0.08944401,
       -0.02222708, -0.09493336, -0.01843699, -0.12703365,  0.14816706,
        0.11948284, -0.49195224,  0.16567157,  0.11348813, -0.07509871,
       -0.19357218,  0.02514981,  0.07136189, -0.04492041, -0.25355104,
       -0.13736355, -0.1256356 ,  0.06755043,  0.00901603, -0.20297523,
       -0.14506412, -0.04295459,  0.03014802, -0.1284308 ,  0.22

Como se explica anteriormente cada palabra estará representada por un vector. La salida que se obtiene de este modelo no puede usarse directamente para alimentar otros modelos, por ello se debe calcular un vector asociado a cada documento.

En la celda que hay a continuación se recorre cada palabra de cada documento, de manera que se vayan sumando los vectores de las palabras que formen la oración. Una vez se consigue la suma total, se divide entre el número de palabras, para así obtener la media de todos los vectores.

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size)) # sumar los vectores de las palabras que contiene cada documento
            count += 1.
        except KeyError:  # manejar el caso de que el token no esté en el vocabulario
            continue
    if count != 0:
        vec /= count# calcular la media de los vectores de las palabras
    return vec

In [ ]:
wordvec_arrays = np.zeros((len(tokenized_tweet), size_vector)) 

for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], size_vector)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(1000, 100)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(reduced_corpus["Tweet_split"], reduced_corpus['Sentiment'], random_state=42, test_size=0.3)

In [ ]:
xtrain_w2v = wordvec_df.iloc[ytrain.index,:]
xvalid_w2v = wordvec_df.iloc[yvalid.index,:]

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [ ]:
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(xtrain_w2v, ytrain)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [ ]:
predictions = clf.predict(xvalid_w2v) 

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_curve, auc 
import concurrent.futures

In [ ]:
def get_metrics(true_labels, predicted_labels):
    
    parameters = []
    
    parameters.append(np.round(metrics.accuracy_score(true_labels,predicted_labels),4))
    parameters.append(np.round(metrics.precision_score(true_labels,predicted_labels,average='weighted'),4))
    parameters.append(np.round(metrics.recall_score(true_labels,predicted_labels,average='weighted'),4))
    parameters.append(np.round(metrics.f1_score(true_labels,predicted_labels,average='weighted'),4) )

    print('Accuracy:', parameters[0])
    print('Precision:', parameters[1])
    print('Recall:', parameters[2])
    print('F1 Score:', parameters[3])
    
    return parameters

In [ ]:
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(true_labels, 
                                           predicted_labels, 
                                           ) 
    
    print(report)
    return report

In [ ]:
def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):

    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(true_labels,predicted_labels)
    cm_frame = pd.DataFrame(cm, pd.MultiIndex([['Predicted:'], classes], level_labels), 
                            pd.MultiIndex([['Actual:'], classes], level_labels)) 
    print(cm_frame)

In [ ]:
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    
    print('Model Performance metrics:')
    print('-'*30)
    parameters = get_metrics(true_labels, predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    report = display_classification_report(true_labels, predicted_labels, classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels, predicted_labels, classes)
    
    

In [ ]:
display_model_performance_metrics(yvalid, predictions,[0,4])

Model Performance metrics:
------------------------------
Accuracy: 0.6133
Precision: 0.6166
Recall: 0.6133
F1 Score: 0.6116

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.64      0.55      0.59       152
           4       0.59      0.68      0.64       148

    accuracy                           0.61       300
   macro avg       0.62      0.61      0.61       300
weighted avg       0.62      0.61      0.61       300


Prediction Confusion Matrix:
------------------------------
             Actual:     
                   0    4
Predicted: 0      83   69
           4      47  101
